#    ATTENDANCE MONITORING SYSTEM USING FACIAL RECOGNITION

In [6]:
cd Documents/IPCV/Attendance Monitoring system

/home/mahadev/Documents/IPCV/Attendance Monitoring system


In [30]:
ls

comparing_images/  students_data/


In [38]:
import cv2
import numpy as np
import face_recognition as fr

#Loading the images and converting it into RGB
imgjoe = fr.load_image_file("comparing_images/elon.png")
imgjoe = cv2.cvtColor(imgjoe,cv2.COLOR_BGR2RGB)
imgtest = fr.load_image_file("comparing_images/bill.png")
imgtest = cv2.cvtColor(imgtest,cv2.COLOR_BGR2RGB)

#Finding faces in our image and finding their encodings
faceLoc = fr.face_locations(imgjoe)[0]
encodejoe = fr.face_encodings(imgjoe)[0]
cv2.rectangle(imgjoe,(faceLoc[3],faceLoc[0],faceLoc[1],faceLoc[2]),(255,0,255),2)

faceLoctest = fr.face_locations(imgtest)[0]
encodetest = fr.face_encodings(imgtest)[0]
cv2.rectangle(imgtest,(faceLoctest[3],faceLoctest[0],faceLoctest[1],faceLoctest[2]),(255,0,255),2)

#Comparing the faces and finding the distance between them(these are 128 measurements of both the faces)
results=fr.compare_faces([encodejoe],encodetest)
facedist = fr.face_distance([encodejoe],encodetest)
print(results,facedist)
cv2.putText(imgtest,f'{results} {round(facedist[0],2)}',(10,25),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(0,0,255),1)


cv2.imshow("elon", imgjoe)
cv2.imshow("bill", imgtest)

cv2.waitKey(0)
cv2.destroyAllWindows()

[False] [0.79390147]


In [39]:
def markattendence(name,usn,dtstring):
    with open('present.csv','r+') as f:
        mydatalist = f.readlines()
        namelist=[]
        for line in mydatalist:
            entry = line.split(',')
            namelist.append(entry[0])
        if name not in namelist:
            now = datetime.now()
            today = now.strftime("%d/%m/%Y")
            f.writelines(f'\n{name},{usn},{dtstring},{today}')
            
def markabsent(absent,absentusn):
    with open('absent.csv','r+') as f:
        mydatalist = f.readlines()
        namelist=[]
        for line in mydatalist:
            entry = line.split(',')
            namelist.append(entry[0])
        for name,usn in zip(absent,absentusn):
            if name not in namelist:
                now = datetime.now()
                today = now.strftime("%d/%m/%Y")
                dtstring = now.strftime('%H:%M:%S')
                f.writelines(f'\n{name},{usn},{dtstring},{today}')
                
                
def Diff(a, b):
    return (list(list(set(a)-set(b))))

In [42]:
import os
import cv2
import numpy as np
import face_recognition as fr
from datetime import datetime
now = datetime.now()
import time

path = 'students_data/'
images = []
classnames = []
mylist = os.listdir(path)
#print(mylist)
for cl in mylist:
    curimg = cv2.imread(f'{path}/{cl}')
    images.append(curimg)
    classnames.append(os.path.splitext(cl)[0])
#print(classnames)

cls=[]
for s in classnames:
    s=s.split('-')
    cls.append(s)
classname,usn=[],[]
for i in cls:
    classname.append(i[0].upper())
    usn.append(i[1].upper())
    

def findencode(images):
    encodelist=[]
    for img in images:
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        encode = fr.face_encodings(img)[0]
        encodelist.append(encode)
    return encodelist


present = []
presentusn = []
present_time = []

elst = findencode(images)
print("encoding complete")


cap = cv2.VideoCapture(0)
t_end = time.time() + 20
while time.time() < t_end:
    success, img = cap.read()
    imgs = cv2.resize(img,(0,0),None,0.25,0.25)
    imgs = cv2.cvtColor(imgs,cv2.COLOR_BGR2RGB)
    
    faces = fr.face_locations(imgs)
    encodes = fr.face_encodings(imgs,faces)
    
    for encodeface,faceloc in zip(encodes,faces):
        matches = fr.compare_faces(elst,encodeface)
        facedis = fr.face_distance(elst,encodeface)
        
        print(facedis)
        matchindex = np.argmin(facedis)
        
        if matches[matchindex]:
            name = classname[matchindex]
            usn1 = usn[matchindex]
            dtstring = now.strftime('%H:%M:%S')
            print(name," - ", usn1)
            y1,x2,y2,x1 = faceloc
            y1,x2,y2,x1 = y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),1)
            cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
            cv2.putText(img,name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(255,255,255),2)
            present.append(name)
            presentusn.append(usn1)
            present_time.append(dtstring)
            
        cv2.imshow('webcam',img)
        cv2.waitKey(1)

cap.release()
cv2.destroyAllWindows()
with open('present.csv','w') as f:
    f.writelines(f'{"NAME"},{"USN"},{"TIME"},{"DATE"}')
    
with open('absent.csv','w') as f:
    f.writelines(f'{"NAME"},{"USN"},{"TIME"},{"DATE"}')

for name, pusn, dtstring in zip(present,presentusn,present_time):
    markattendence(name, pusn, dtstring)
    
absent = Diff(classname, present)
absentusn = Diff(usn, presentusn)
markabsent(absent,absentusn)

encoding complete
[0.77726623 0.46384887 0.60514963]
MAHADEV  -  2GI18EC055
[0.79706066 0.46389575 0.62489571]
MAHADEV  -  2GI18EC055
[0.76942756 0.4839064  0.60901357]
MAHADEV  -  2GI18EC055
[0.75124616 0.45832606 0.59794142]
MAHADEV  -  2GI18EC055
[0.76198192 0.46678829 0.60123374]
MAHADEV  -  2GI18EC055
[0.75632297 0.49875071 0.62197405]
MAHADEV  -  2GI18EC055
[0.73507997 0.51738697 0.67575179]
MAHADEV  -  2GI18EC055
[0.75107945 0.54670137 0.64749533]
MAHADEV  -  2GI18EC055
[0.73583413 0.53670892 0.62277526]
MAHADEV  -  2GI18EC055
[0.80949862 0.55825671 0.63957828]
MAHADEV  -  2GI18EC055
[0.79188489 0.53690463 0.58387882]
MAHADEV  -  2GI18EC055
[0.77956679 0.50911443 0.57509048]
MAHADEV  -  2GI18EC055
[0.77322068 0.50034213 0.53931372]
MAHADEV  -  2GI18EC055
[0.75089758 0.508511   0.52560203]
MAHADEV  -  2GI18EC055
[0.76227403 0.49662219 0.57076697]
MAHADEV  -  2GI18EC055
[0.77723065 0.50504969 0.62882825]
MAHADEV  -  2GI18EC055
[0.78519546 0.46336123 0.61401319]
MAHADEV  -  2GI18EC